In [6]:
import torch
print(torch.__version__)

# cu가 안뜨면 CPU버전이다.

2.3.1+cu121


In [7]:
import sys
print(sys.version)

# 3.13 버전 사용 X

3.10.12 (main, Nov  4 2025, 08:48:33) [GCC 11.4.0]


In [8]:
# CUDA(GPU) 사용 가능 여부 확인 -->주석자체가 프롬프트다.
# 셀에 주석을 잘 걸어놓으면 좋은 코드가 나온다.
import torch
print(torch.cuda.is_available()) # return True or False --> 조건식 OK

# GPU 이름 확인
print(torch.cuda.get_device_name(0))


True
NVIDIA GeForce GTX 1660 SUPER


In [9]:
# 3. GPU가 사용 가능한 경우에만, 관련된 상세 정보 출력
# 코드 기억해야함
# 파란색 function 다 외워야 함
if torch.cuda.is_available():
    print("사용가능한 GPU 수 :", torch.cuda.device_count()) # 현장가면 엄청 많음
    current_gpu_index = torch.cuda.current_device()
    print("현재 사용 중인 GPU 인덱스 :", current_gpu_index) # GPU가 여러개일땐 인덱스 변경 가능, 분산처리도 가능
    print('Current GPU Name:', torch.cuda.get_device_name(current_gpu_index))
    print('Current GPU Memory:', torch.cuda.get_device_properties(current_gpu_index).total_memory / 1e9, "GB")
    print(f'\CUDA version: {torch.version.cuda}')

else:
    print("GPU가 사용 불가능합니다.")

#GPU의 메모리 사용량 확인
#중요한건 :AI를 제어하는 개발자가 되야함, 그래야 좋은 개발자임,




사용가능한 GPU 수 : 1
현재 사용 중인 GPU 인덱스 : 0
Current GPU Name: NVIDIA GeForce GTX 1660 SUPER
Current GPU Memory: 6.441992192 GB
\CUDA version: 12.1


In [10]:
#4.간단한 텐서(Tensor) 생성 및 출력 
import torch    
x=torch.tensor([1,2,3]) # x는 변수: tensor라는 opject를 가지는 변수이다.
                        # 텐서: 다차원배열(array)근데 nadarty는 아니다
                    
print(x)
print(f'Type of x: {x.type()}')
# print(f'Type of x: {x.dtype()}' --> 에러발생함
print(f'Type of x: {x.dtype}')  # dtype은 원래 넘파이에 나오지만 다르다,device=CPU
print(f'Size of x: {x.size()}')
# print할때는cpu쓸수밖에 없음,
print(f'Device of x: {x.device}')

# 데이터가 일부는 cpu,일부는 gpu에 저장하면 학습할때 멈출수 있어서 위에 코드 필요함

tensor([1, 2, 3])
Type of x: torch.LongTensor
Type of x: torch.int64
Size of x: torch.Size([3])
Device of x: cpu


In [11]:
# 1-2 텐서를 GPU로 보내고 다시 가져오기
# tensor를 다루려면 torch 가 필요하다.
import time # 파이썬 내장용 , 원래 내장모듈을 먼저 import하는게 빠르다.
import torch # 파이외장 모듈
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# import torch.utils.data

In [12]:
# 1. 장치(device)설정:GPU가 있으면,'cuda'를 없으면 'cpu'를 사용 
# 여러개의 GPU가 있을경우엔 'cuda:0','cuda:1' 같은 식으로 지정 가능
# 현재코드는 GPU가 한개인 경우로 가정 --> 'cuda'로 설정

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}") # cuda 로 나왔다 결국 gpu 다

Using device: cuda


In [13]:
# 2 텐서 생성: cpu생성(기본동작)
cpu_tensor = torch.tensor([ # 오브젝트 구성: attribute & method
    [1,2,3],
    [4,5,6]
]
,dtype=torch.float16 # 중요한 이유:  16으로 바꾸면 메모리가 2배로 줄어듭니다.
                    # 근데 8은 오류남, 최소데이터수가 16임 / LLM 에서 되게 중요함
)
print(cpu_tensor) # 텐서 오브젝트 생성
print(cpu_tensor.dtype) # 만약 1.0으로 고치면 전체 배열이 소수로 바뀜,float32로 출력 / 근데 이렇게 맡기면 안되고 dtpye직접 기정해야함

print(cpu_tensor.device) # 얘가 어디에 저장되어 있는지 확인



tensor([[1., 2., 3.],
        [4., 5., 6.]], dtype=torch.float16)
torch.float16
cpu


In [14]:
# cpu -> gpu 로 옮기기
start_time = time.time()
moved_tensor = cpu_tensor.to(device) # Gpu로 보내라(복사해서 보낸다.) , RAM --> VRAM으로 복사해서 보내라(삭제가 X)
end_time = time.time()

print(moved_tensor) # 그냥 쿠다가 아니라 device='cuda:0' 이렇게 나옴

print(f'이동 시간: {end_time - start_time:.5f}초') # 짧은시간에 옮겨졌다. 근데 시간은 걸린다 ,작은데이터라 / 큰데이터는 시간 더 걸림
# 즉 한방에 데이터가 슥 옮겨질수 없다.
print(cpu_tensor) # device 안나옴, 즉 아직 cpu에 남아있다. 

tensor([[1., 2., 3.],
        [4., 5., 6.]], device='cuda:0', dtype=torch.float16)
이동 시간: 0.36204초
tensor([[1., 2., 3.],
        [4., 5., 6.]], dtype=torch.float16)


In [15]:
# GPU에서 직접 텐서 생성 가능
# 이방법은 CPU 메모리를 거치지 않고 바로 GPU 메모리에 텐서 할당하여 더 효율적일 수 있다.
if torch.cuda.is_available():
    # GPU 사용 가능
    start_time = time.time()
    direct_gpu_tensor = torch.ones(2,3, device=device) # torch.ones()은 CPU 메모리에 텐서를 생성하고 device로 이동
    end_time = time.time()
    print(direct_gpu_tensor)
    print(f"GPU 텐서 생성 시간: {end_time - start_time:.5f} 초") # 위에 cpu->gpu할때보다 속도 더 빨라짐
else:
    # GPU 사용 불가능
    pass



tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
GPU 텐서 생성 시간: 0.01297 초


In [16]:
# 이동 시간: 0.00124초
# 직접 생성 :0.00091 초 
# gpu -> cpu  : 다시CPU로 옮기기
cpu_to_tensor = moved_tensor.cpu() # method
print(cpu_to_tensor.device)


cpu


In [ ]:
# 텐서 중요 속성 다루기
# 교재에 있는 코드는 의미는 없지만 속성들은 중요하니까 봐야함
# 3X5 크기의 float32 타입의 텐서 생성,gpu 사용,미분가능
# 텐서단위로 미분할 수 있다!!!
import torch
tensor_ = torch.randn(3,5,dtype=torch.float32,device=device, requires_grad=True)  # 신경망에 바로 들어가 사용 가능

print(tensor_.requires_grad)
print(tensor_.is_leaf) # 현재는 의미없다. 왜 ? 하나밖에 없어서 이미 나무에 잎이 하나밖에 없어서, 나중에 끝을 따질때가 있다.....
                       # 그리고 leaf 가 여러개 있을수도 있다.
                       # 

# 계산그래프:텐서와 텐서를 연산하도록 하는것

    


True
True
